In [21]:
from sqlalchemy import create_engine
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from env_vars import *
import pandas as pd
import sqlite3
from sqlalchemy import engine
import pickle
import numpy as np

In [3]:
def authorize():
    client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_features(trackID):
        return sp.audio_features(trackID)[0]
    
def get_release_date(trackID):
    song_release_date = sp.track(trackID)['album']['release_date']
    return song_release_date

In [ ]:
df = pickle.load( open( "./song_list_v5.pkl", "rb"))

In [ ]:
engine = create_engine('sqlite:///song_list_curr.db', echo=False)
df.to_sql('songs', con=engine)
engine.execute('SELECT * FROM songs;').fetchone()

In [4]:
for row_index in hashed_df['songid'][0:574019]:
    songid = row_index
    #print(songid)
    date = get_release_date(songid)
    #print(markets)
    try:
        engine.execute(f'UPDATE songs SET availability = "{date}" WHERE songid = \"{songid}\"')
        authorize()
    except:
        print(f"couldn't insert {date} into table")
        continue

In [27]:
df['song_release_date'] =  pd.to_datetime(df['song_release_date'])

In [28]:
df.dtypes

col1                          int64
song_release_date    datetime64[ns]
dtype: object

In [29]:
df

,col1,song_release_date
0,1,1977-10-28
1,2,1997-01-01
